In [ ]:
# import simulation stuff
import conf_solo12_trot_step_adjustment_full_kinematics as conf
from robot_properties_solo.solo12wrapper import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from bullet_utils.env import BulletEnvWithGround
from simulate_solo import Simulator

# initialize robot in pybullet
env = BulletEnvWithGround()
simulator = Simulator(env, Solo12Robot(), conf)
robot = Solo12Config.buildRobotWrapper()
q0, dq0 = simulator.q0, simulator.nu0
simulator.robot.reset_state(q0, dq0) 
simulator.robot.pin_robot.framesForwardKinematics(q0)

step_length = conf.gait['stepLength']
w = 0.5*conf.step_adjustment_bound
print(len(conf.contact_sequence))
for i, c in enumerate(conf.contact_sequence):
    for debri in c:
        if debri.ACTIVE:
            p = debri.pose.translation
            simulator.build_one_stepstone(
                start_pos=(p[0]-w, p[1], p[2]-0.017), orientation=(0., 0., 0, 1),
                stone_length=conf.step_adjustment_bound, stone_width=conf.step_adjustment_bound   
    )

In [ ]:
# import models and solvers
from centroidal_plus_double_integrator_kinematics_acados_solver import CentroidalPlusLegKinematicsAcadosSolver
from centroidal_plus_double_integrator_kinematics_casadi_model  import CentroidalPlusLegKinematicsCasadiModel
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
import numpy as np

# DDP warm-start
wbd_model = WholeBodyModel(conf)
ddp_planner = WholeBodyDDPSolver(wbd_model, MPC=False, WARM_START=False)
ddp_planner.solve()
ddp_sol = ddp_planner.get_solution_trajectories()
centroidal_warmstart = ddp_sol['centroidal']
q_warmstart = ddp_sol['jointPos']
qdot_warmstart = ddp_sol['jointVel']
x_warmstart = []
u_warmstart = []
rmodel, rdata = conf.rmodel, conf.rdata
for k in range(len(centroidal_warmstart)):
    x_warmstart.append(
        np.concatenate(
            [centroidal_warmstart[k],
             q_warmstart[k], 
             qdot_warmstart[k]]
            )
        )
    u_warmstart.append(np.concatenate([np.zeros(30)]))
# nominal traj-opt
model_nom = CentroidalPlusLegKinematicsCasadiModel(conf, STOCHASTIC_OCP=True)
solver_nom = CentroidalPlusLegKinematicsAcadosSolver(
    model_nom, x_warmstart, u_warmstart, MPC=False
)
x_nom, u_nom = solver_nom.solve()

In [ ]:
# run monte-carlo simulations
from centroidal_plus_double_integrator_kinematics_acados_simulator import CentroidalPlusLegKinematicsAcadosSimulator
nb_sims = 10
simulator = CentroidalPlusLegKinematicsAcadosSimulator(model_nom, solver_nom)
x_sim = simulator.simulate(x_nom[0, :], u_nom, nb_sims=nb_sims, WITH_DISTURBANCES=True)
violations = simulator.count_constraint_violations(x_sim, WITH_VISUALIZATION=True)
print(violations)